In [80]:
import smact.lattice as lattice
import smact.builder as builder
import smact.core as core
import copy
import os
import csv
import numpy as np

In [31]:
# get correct path for datafiles when called from another directory
smact_directory = '../../smact'
# Append a trailing slash to make coherent directory name - this would select the
# root directory in the case of no prefix, so we need to check
if smact_directory:
    smact_directory = smact_directory + '/'

### Set up a lattice object

In [62]:
site_A = lattice.Site([0,0,0],[+1,+2,+3])
site_B = lattice.Site([0.5,0.5,0.5],[+5,+4,+3,+2])
site_C = lattice.Site([0.5,0.5,0.5],[-2,-1])
perovskite = lattice.Lattice([site_A,site_B,site_C],space_group=221)

### Set up a search space with Josh's command

In [63]:
search = core.ordered_elements(3,37)

###Assign potential elements to each site, based on known coordination environments
* A site : 12 coordinated
* B site : 8 coordinated
* The third item in the list for each species is the Shannon radius for that charge state

In [76]:
A_list = []
B_list = []
C_list = [['O',-2,1.35],['S',-2,1.84],['Se',-2,1.98],['F',-1,1.285],['Br',-1,1.96],['I',-1,2.2]]
for element in search:
    with open(smact_directory + 'data/shannon_radii.csv','rU') as f:
        reader = csv.reader(f)
        r_shannon=False
        for row in reader:
            if row[2]=="12_n" and row[0]==element and int(row[1]) in site_A.oxidation_states:
                A_list.append([row[0],row[1],row[4]])
            if row[2]=="8_n" and row[0]==element and int(row[1]) in site_B.oxidation_states:
                B_list.append([row[0],row[1],row[4]])

In [85]:
charge_balanced = []
goldschmidt_cubic = []
goldschmidt_ortho = []
a_too_large = []
A_B_similar = []

for A in A_list:
    for B in B_list:
        for C in C_list:
            if int(A[1])+int(B[1])+3*int(C[1]) == 0:
                charge_balanced.append([A[0],B[0],C[0]])
                tol = (float(A[2]) + C[2])/(np.sqrt(2)*(float(B[2])+C[2]))
                if tol > 1.0:
                    a_too_large.append([A[0],B[0],C[0]])
                if tol > 0.9 and tol <= 1.0:
                    goldschmidt_cubic.append([A[0],B[0],C[0]])
                if tol >= 0.71 and tol < 0.9:
                    goldschmidt_ortho.append([A[0],B[0],C[0]])
                if tol < 0.71:
                    A_B_similar.append([A[0],B[0],C[0]])
                    

In [86]:
print 'Number of possible charge neutral perovskites from', search[0], 'to', search[len(search)-1], '=', len(charge_balanced)
print 'Number of possible cubic perovskites from', search[0], 'to', search[len(search)-1], '=', len(goldschmidt_cubic)
print 'Number of possible ortho perovskites from', search[0], 'to', search[len(search)-1], '=', len(goldschmidt_ortho)
print 'Number of possible hexagonal perovskites from', search[0], 'to', search[len(search)-1], '=', len(a_too_large)
print 'Number of possible non-perovskites from', search[0], 'to', search[len(search)-1], '=', len(A_B_similar)

Number of possible charge neutral perovskites from Li to Kr = 48
Number of possible cubic perovskites from Li to Kr = 9
Number of possible ortho perovskites from Li to Kr = 35
Number of possible hexagonal perovskites from Li to Kr = 4
Number of possible non-perovskites from Li to Kr = 0


In [83]:
print charge_balanced


[['Na', 'Mg', 'F'], ['Na', 'Mg', 'Br'], ['Na', 'Mg', 'I'], ['Na', 'Ca', 'F'], ['Na', 'Ca', 'Br'], ['Na', 'Ca', 'I'], ['Na', 'Cr', 'O'], ['Na', 'Cr', 'S'], ['Na', 'Cr', 'Se'], ['Na', 'Mn', 'F'], ['Na', 'Mn', 'Br'], ['Na', 'Mn', 'I'], ['Na', 'Fe', 'F'], ['Na', 'Fe', 'Br'], ['Na', 'Fe', 'I'], ['Na', 'Co', 'F'], ['Na', 'Co', 'Br'], ['Na', 'Co', 'I'], ['Na', 'Zn', 'F'], ['Na', 'Zn', 'Br'], ['Na', 'Zn', 'I'], ['K', 'Mg', 'F'], ['K', 'Mg', 'Br'], ['K', 'Mg', 'I'], ['K', 'Ca', 'F'], ['K', 'Ca', 'Br'], ['K', 'Ca', 'I'], ['K', 'Cr', 'O'], ['K', 'Cr', 'S'], ['K', 'Cr', 'Se'], ['K', 'Mn', 'F'], ['K', 'Mn', 'Br'], ['K', 'Mn', 'I'], ['K', 'Fe', 'F'], ['K', 'Fe', 'Br'], ['K', 'Fe', 'I'], ['K', 'Co', 'F'], ['K', 'Co', 'Br'], ['K', 'Co', 'I'], ['K', 'Zn', 'F'], ['K', 'Zn', 'Br'], ['K', 'Zn', 'I'], ['Ca', 'Ti', 'O'], ['Ca', 'Ti', 'S'], ['Ca', 'Ti', 'Se'], ['Ca', 'V', 'O'], ['Ca', 'V', 'S'], ['Ca', 'V', 'Se']]
